In [24]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Leitura de instâncias

In [25]:
def read_dat_file(file_path):
    """"Função para a leitura das instâncias de Mariá"""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 1. Lendo quantidade de itens e períodos
    items, periods = map(int, lines[0].split())

    # 2. Lendo número de plantas
    num_plants = int(lines[1].strip())

    # 3. Lendo capacidades das plantas
    capacities = [int(lines[i + 2].strip()) for i in range(num_plants)]
    capacities = np.tile(capacities, (periods, 1)).T  # Repete as capacidades ao longo dos períodos (deixar na forma j, t)

    # 4. Lendo a matriz de produção (tempo de produção, tempo de setup, custo de setup, custo de produção)
    production_data = []
    start_line = 2 + num_plants
    production_time = np.zeros((items, num_plants))  # Inicializar listas para armazenar separadamente os tempos e custos
    setup_time = np.zeros((items, num_plants))
    setup_cost = np.zeros((items, num_plants))
    production_cost = np.zeros((items, num_plants))
    for i in range(num_plants * items):  # Preencher as matrizes com os dados lidos
        plant = i // items  # Determina a planta
        item = i % items    # Determina o item
        # Extrair os dados de cada linha
        prod_time, set_time, set_cost, prod_cost = map(float, lines[start_line + i].split())
        production_time[item, plant] = prod_time  # Preencher as respectivas matrizes
        setup_time[item, plant] = set_time
        setup_cost[item, plant] = set_cost
        production_cost[item, plant] = prod_cost

    # 5. Lendo os custos de inventário
    inventory_costs_line = start_line + num_plants * items
    inventory_costs = list(map(float, lines[inventory_costs_line].split()))  # Lê todos os valores de inventory_costs como uma única lista
    inventory_costs = np.array(inventory_costs).reshape(num_plants, -1)  # Divide a lista de custos de inventário por planta
    inventory_costs = inventory_costs.T  # Deixa na forma (i, j)

    # 6. Lendo a matriz de demanda (12 linhas, 12 colunas)
    demand_matrix = []
    demand_start_line = inventory_costs_line + 1
    
    # Leitura inicial das demandas
    for i in range(periods):  # Lê as linhas de demandas para os períodos
        demands = list(map(int, lines[demand_start_line + i].split()))
        demand_matrix.append(demands)
    
    # Agora vamos dividir os valores de cada linha combinada entre as plantas
    final_demand_matrix = []
    for demands in demand_matrix:
        period_demand = []
        for j in range(num_plants):
            # Divide a demanda combinada por planta, assumindo que cada planta tem o mesmo número de itens
            plant_demand = demands[j*items:(j+1)*items]
            period_demand.append(plant_demand)
        final_demand_matrix.append(period_demand)
    
    # Transpor a matriz de demanda para o formato correto (itens, plantas, períodos)
    final_demand_matrix = np.array(final_demand_matrix)
    final_demand_matrix = np.transpose(final_demand_matrix, (2, 1, 0))  # Converte para o formato (itens, plantas, períodos)

    # 7. Reading transfer costs directly from the document as a matrix
    transfer_cost_matrix = []
    transfer_cost_line = demand_start_line + periods

    # Read the matrix of transfer costs line by line
    while transfer_cost_line < len(lines):
        line = lines[transfer_cost_line].strip()
        if line:
            # Split the line into individual cost values and convert them to float
            row = [float(value) for value in line.split()]
            transfer_cost_matrix.append(row)
        transfer_cost_line += 1

    # Convert to a numpy array (optional, if you want to work with numpy for matrix operations)
    transfer_costs = np.array(transfer_cost_matrix)

    return {"items": items,
            "periods": periods,
            "num_plants": num_plants,
            "capacities": capacities,
            "production_time": production_time,
            "setup_time": setup_time,
            "setup_cost": setup_cost,  
            "production_cost": production_cost,
            "inventory_costs": inventory_costs,
            "demand_matrix": final_demand_matrix,
            "transfer_costs": transfer_costs}

In [26]:
# Exemplo de uso
file_path = '../instancias/multi_plant_instances/AAA00_12_2_10.dat'
data = read_dat_file(file_path)
display(data)

{'items': 10,
 'periods': 12,
 'num_plants': 2,
 'capacities': array([[3069, 3069, 3069, 3069, 3069, 3069, 3069, 3069, 3069, 3069, 3069,
         3069],
        [2804, 2804, 2804, 2804, 2804, 2804, 2804, 2804, 2804, 2804, 2804,
         2804]]),
 'production_time': array([[2.4, 1.3],
        [3.8, 4.1],
        [1.8, 2.6],
        [2.2, 3.2],
        [4.4, 3.6],
        [1.6, 1.9],
        [3.8, 4.6],
        [4.7, 4.7],
        [4.2, 2.2],
        [4.9, 1.1]]),
 'setup_time': array([[69.1, 66.4],
        [23.8, 48.7],
        [23.3, 74.5],
        [38.6, 16.8],
        [48.9, 60.8],
        [58.1, 67.8],
        [71.5, 51.6],
        [57.6, 65.5],
        [22.4, 68.5],
        [20.6, 44.8]]),
 'setup_cost': array([[561.8, 139.1],
        [946.3, 871.3],
        [898.8,  55.2],
        [419.1, 918.5],
        [604.3, 937.9],
        [643.1, 593.9],
        [ 91.2, 143.3],
        [821.9, 208.5],
        [ 65. , 667.6],
        [578.8, 221.5]]),
 'production_cost': array([[2.1, 1.6],
  

# Modelagem

In [27]:
m = gp.Model('Lot-sizing Sambasivan and Yahya')

## Conjuntos

In [28]:
# Produtos (i)
I = np.array([_ for _ in range(data['items'])])
# Plantas (j)
J = np.array([_ for _ in range(data['num_plants'])])
# Períodos (t)
T = np.array([_ for _ in range(data['periods'])])

## Parâmetros

In [29]:
# Demanda (i, j, t)
d = np.array(data['demand_matrix'])
# Capacidade (j, t)
cap = np.array(data['capacities'])
# Tempo de produção (i, j)
b = np.array(data['production_time'])
# Tempo de setup (i, j)
f = np.array(data['setup_time'])
# Custo de produção (i, j)
c = np.array(data['production_cost'])
# Custo de setup (i, j)
s = np.array(data['setup_cost'])
# Custo de transporte (j, k)
r = np.array(data['transfer_costs'])
# Custo de estoque (i, j)
h = np.array(data['inventory_costs'])

## Variáveis de decisão

In [30]:
# Quantidade produzida (i, j, t)
X = m.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='X')
# Quantidade estocada (i, j, t)
Q = m.addVars(I, J, T, vtype=GRB.CONTINUOUS, name='Q')
# # Quantidade transportada (i, j, k(um outro j), t)
W = m.addVars(I, J, J, T, vtype=GRB.CONTINUOUS, name='W')
# # Variável de setup (binária) (i, j, t)
Z = m.addVars(I, J, T, vtype=GRB.BINARY, name='Z')

## Função objetivo

In [31]:
expr_objetivo = sum(sum(sum(c[i, j] * X[i, j, t] + h[i, j] * Q[i, j, t] + s[i, j] * Z[i, j, t] + 
                            sum(r[j, k] * W[i, j, k, t] for k in J if k != j) for t in T) for j in J) for i in I)
m.setObjective(expr_objetivo, sense=GRB.MINIMIZE)

## Restrições

In [32]:
# Balanço de estoque (revisar comportamento)
# Período inicial
m.addConstrs((Q[i, j, t] == X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t == 0),
             name='restricao_balanco_estoque')
# Demais períodos
m.addConstrs((Q[i, j, t] == Q[i, j, t-1] + X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t > 0),
             name='restricao_balanco_estoque');

In [33]:
# Restrição que obriga setup (validar o range do r)
m.addConstrs((X[i, j, t] <= min((cap[j, t] - f[i, j]) / b[i, j], sum(sum(d[i, k, r] for r in range(t, T[-1] + 1)) for k in J)) * Z[i, j, t] for i in I for j in J for t in T)
             , name='restricao_setup');
# m.addConstrs((X[i, j, t] <= sum(sum(d[i, j, b] for b in range(t, len(T))) for j in J) * Z[i, j, t] for i in I for j in J for t in T)
#              , name='restricao_setup');

In [34]:
# Restrição de capacidade
m.addConstrs((sum(b[i, j] * X[i, j, t] + f[i, j] * Z[i, j, t] for i in I) <= cap[j, t] for j in J for t in T)
             , name='restricao_capacidade');

# Resolução

In [35]:
# Critérios de parada
m.Params.timelimit = 60  # Tempo (default = inf)
# m.Params.MIPgap = 0.01  # Gap de otimalidade (default = 0.0001 = 0.01%)

# Otimização
m.optimize()

Set parameter TimeLimit to value 60
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 504 rows, 1200 columns and 2140 nonzeros
Model fingerprint: 0xba7263df
Variable types: 0 continuous, 1200 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e-01, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 0 rows and 260 columns
Presolve time: 0.00s
Presolved: 504 rows, 940 columns, 2120 nonzeros
Variable types: 0 continuous, 940 integer (240 binary)

Root relaxation: objective 5.026663e+04, 996 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Tim

In [36]:
# Número de períodos da instância
len(T)

12

In [37]:
# Número de fábricas
len(J)

2

In [38]:
# Número de produtos
len(I)

10

In [39]:
# Tempo de execução em segundos
m.Runtime

60.037999868392944

In [40]:
# Nome do modelo
m.ModelName

'Lot-sizing Sambasivan and Yahya'

In [41]:
# Valor da função objetivo da melhor solução encontrada (upper bound da minimização)
m.ObjVal

64522.96000000001

In [42]:
# Lower bound da minimização
m.ObjBound

62933.96204801117

In [43]:
# Gap da solução
m.MIPgap

0.024626860763809294

In [44]:
m.Status  # Ver https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes para interpretar códigos

9